In [28]:
import os
import requests
import json
import re
from dotenv import load_dotenv
from langchain.chains import LLMChain
from faker import Faker


In [57]:
load_dotenv()

LLAMAV3_API_KEY = os.environ["LLAMAV3_API_KEY"]
if LLAMV3_API_KEY is None:
    raise ValueError("LLAMV3_API_KEY does not exist, add it to env")

prompt = ("Du är en säkerhetsassistent."
          "Din uppgift är att identifiera all personligt identifierbar information (PII)."
          "Lista endast känslig information och inget annat."
          "Följ den strukturerade formatet nedan när du rapporterar upptäckt känslig information. Inkludera inte '*'."
          "Det är avgörande att denna struktur följs noggrant för att upprätthålla datans integritet:"

          "Namn: [Ange fullständigt namn här]"
          "Telefonnummer: [Ange telefonnummer här]"
          "E-postadress: [Ange e-postadress här]"
          "Adress: [Ange fullständig adress här]"
          "Bankkortsnummer: [Ange bankkortsnummer här]"
          "Ålder: [Ange ålder här]"
          "Personnummer: [Ange personnummer]"
          
          "Om du stöter på personligt identifierbar information (PII) som inte direkt matchar någon av dessa etiketter,"
          "hitta den närmast motsvarande etiketten och använd den. Se till att alla poster är korrekta och kompletta.")

def query(prompt):
    response = requests.post(
      url="https://openrouter.ai/api/v1/chat/completions",
      headers={
        "Authorization": f"Bearer {LLAMAV3_API_KEY}",
      },
      data=json.dumps({
        "model": "meta-llama/llama-3-8b-instruct",
        "messages": [
          {"role": "user", "content": prompt}
        ]
      })
    )
    response_data = response.json()
    return response_data['choices'][0]['message']['content']

def parse_lmm_output(text):
    pii = {}
    lines = text.strip().split("\n")
    for line in lines:
        if ": " in line:
            key, value = line.split(": ", 1)
            pii[key] = value  # Remove extra space and punctuation from keys
    return pii

def generate_dummy_data(text, pii_data):
    for label, value in pii_data.items():
        #print(f"Replacing {label} with value {value}")  # Debugging print
        if label == "Namn":
            text = text.replace(value, fake.name())
        elif label == "Telefonnummer":
            text = text.replace(value, fake.phone_number())
        elif label == "E-postadress":
            text = text.replace(value, fake.email())
        elif label == "Adress":
            text = text.replace(value, fake.address())
        elif label == "Bankkortsnummer":
            text = text.replace(value, fake.credit_card_number())
        elif label == "Ålder":
            text = text.replace(value, "20")
        elif label == "Personnummer":
            text = text.replace(value, fake.credit_card_number())
        else:
            text = text.replace(value, "[Anonymized]")
    return text

def read_text_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"The file {file_path} was not found.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


def main_process(file_path):
    text_content = read_text_file(file_path)
    if text_content:
        combined_text = prompt + " " + text_content
        lmm_output = query(combined_text)
        print("Detected Sensitive Information in Text:")
        print(lmm_output)
        pii_data = parse_lmm_output(lmm_output)
        anonymized_text = generate_dummy_data(text_content, pii_data)
        print("\nAnonymized Text:")
        print(anonymized_text)
    else:
        print("No text was processed.")
    

file_path = input("Enter the path to your text file: ")
main_process(file_path)


Enter the path to your text file:  test.txt


Detected Sensitive Information in Text:
Namn: Felix Böhlin
Telefonnummer: 076-1234567
E-postadress: felix.2000@gmail.com
Adress: Västerlånggatan 24, 19164 Sollentuna
Bankkortsnummer: 1234-1234-1234-1234
Ålder: 23

Anonymized Text:
hej, jag heter Elisabet Carlsson och jag är 20 år gammal. Du kan nå mig på 031-60 22 42 eller emma19@example.net och jag bor i Sollentuna 19164 på Västerlånggatan 24. mitt kort är 3557850521513407. 
